In [1]:
from rank_bm25 import BM25Okapi
from tqdm import tqdm
from nltk.tokenize import word_tokenize

import numpy as np
import json
from bs4 import BeautifulSoup
import torch
import gzip

from transformers import DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoderTokenizer
from transformers import DPRQuestionEncoder
from transformers import DPRContextEncoder
import csv
from transformers import BertModel, BertTokenizer, BertTokenizerFast
from torch.nn import CosineSimilarity
from torch.utils.data import DataLoader, Dataset
from nltk import word_tokenize
import pandas as pd
import random

from IPython import embed
from sklearn.metrics import classification_report
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print(device)
torch.cuda.empty_cache()

cuda


In [3]:
class SimpleDataset(Dataset):
    def __init__(self, context_file, question_file, idxs):
        context_vals = torch.load(context_file) #change these file lines depending on which dataset you want to load from NQ
        question_vals = torch.load(question_file)
        self.context_embeds = []
        for elem in context_vals:
            self.context_embeds.append(torch.reshape(elem, (1, 768)))
        self.question_embeds = []
        for elem in question_vals:
            self.question_embeds.append(torch.reshape(elem, (1, 768)))
        self.idxs = idxs

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, idx):
        batch_ixs = self.idxs[idx]
        query_embed = self.question_embeds[   int(batch_ixs[0])   ]
        context_embeds = [ self.context_embeds[int(batch_ixs[i])]  for i in range(len(batch_ixs) -1)  ]
        context_embeds[0] = self.context_embeds[ int(batch_ixs[0]) ]
        label = idx
        return (query_embed, context_embeds, batch_ixs[0:-1], label)


In [5]:
def run_DPR(query_embed, context_embeds, ixs):
    similarities = []
    for i in range(len(context_embeds)):
        similarities.append( torch.dot(query_embed[0], context_embeds[i][0])/torch.norm(context_embeds[i][0]) )
        #embed()
        #a = normalize(e)
        #b = normalize(query_embed)
        
        #similarities.append( CosineSimilarity(dim=-1, eps=1e-6)(e, query_embed)  )
    #embed()
    return int(ixs[np.argmax(np.array(similarities))])  #, dim = 0)

In [11]:
def NQ_processing(idx_f):
    
    rows = []
    with open(idx_f, newline='') as idx_file:
        for line in csv.reader(idx_file):
            rows.append(list(line))
    idx_file.close()
    return rows
    

In [1]:
#For Training Datasets
i = 1
question_file = f"/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/train/question-{i}-embeds"
context_file = f"/home/ubuntu/nlm/williamyang/Data/NQ/train/nq-train-0{i}.jsonl.gz"
idx_file = f"/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/train/nq-train-ix-{i}.csv"
phrase_len = 50

In [10]:
rows = NQ_processing(idx_file)
dataset = SimpleDataset(context_file, question_file, rows)
dataloader = DataLoader(dataset, batch_size=50)